In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
import torch

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
def get_embedding(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)

    # Get the model's output
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the last hidden state and apply mean pooling
    last_hidden_states = outputs.last_hidden_state

    # Create an attention mask to ignore padding tokens
    attention_mask = inputs['attention_mask']

    # Apply mean pooling
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return sum_embeddings / sum_mask

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset.csv')
df.head()

,Title,Author,Genres,Summary,word_count,absurdist fiction,adventure,adventure novel,albino bias,alien invasion,...,utopian fiction,vampire fiction,war novel,western,western fiction,whodunit,wuxia,young adult literature,zombie,zombies in popular culture
0,Animal Farm,George Orwell,"['satire', 'speculative fiction', 'roman à cle...","Old Major, the old boar on the Manor Farm, ca...",957.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A Clockwork Orange,Anthony Burgess,"['novella', 'utopian and dystopian fiction', '...","Alex, a teenager living in near-future Englan...",998.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,The Plague,Albert Camus,"['fiction', 'absurdist fiction', 'existentiali...",The text of The Plague is divided into five p...,1119.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A Fire Upon the Deep,Vernor Vinge,"['hard science fiction', 'fantasy', 'speculati...",The novel posits that space around the Milky ...,722.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,All Quiet on the Western Front,Erich Maria Remarque,"['war novel', 'roman à clef']","The book tells the story of Paul Bäumer, a Ge...",770.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import numpy as np

embeddings = []
for index, row in df.iterrows():
    # to handle potential missing summaries
    if pd.notna(row['Summary']) and len(row['Summary']) > 0:
        embedding = get_embedding(row['Summary']).squeeze().numpy()
        embeddings.append(embedding)
    else:
        # apend a placeholder for books with no valid summary
        embeddings.append(np.full(768, 0.0))

# convert the list of embeddings to a numpy array
embeddings_array = np.array(embeddings)

In [ ]:
np.save('drive/My Drive/Colab Notebooks/book_embeddings.npy', embeddings_array)